## Demand Forecasting Using Prophet on Quik Data

In [62]:
import numpy as np
import pandas as pd
import pandasql as ps
import math
import itertools 
from scipy import stats
import time
import pickle
from openpyxl import Workbook


#for data cleaning
import os
import csv
import json

#matplotlib libraries
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors
import seaborn as sns

#date libraries
from dateutil import parser
from datetime import datetime, timedelta, date
import holidays

#prophet library
from prophet import Prophet
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_cross_validation_metric
from prophet.diagnostics import cross_validation, performance_metrics

#pandas options
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#matplotlib setting defaults
sns.set(font="Arial",
        rc={
 "axes.axisbelow": False,
 "axes.edgecolor": "lightgrey",
 "axes.facecolor": "None",
 "axes.grid": False,
 "axes.labelcolor": "dimgrey",
 "axes.spines.right": False,
 "axes.spines.top": False,
 "figure.facecolor": "white",
 "lines.solid_capstyle": "round",
 "patch.edgecolor": "w",
 "patch.force_edgecolor": True,
 "text.color": "dimgrey",
 "xtick.bottom": False,
 "xtick.color": "dimgrey",
 "xtick.direction": "out",
 "xtick.top": False,
 "ytick.color": "dimgrey",
 "ytick.direction": "out",
 "ytick.left": False,
 "ytick.right": False})

### Assumptions For Historical Data Processing:
1. The unprocessed historical data stored in a folder og_data
2. The files inside og_data are all .csv files
3. Columns have the same names as given: 'order_id', 'merchant_id', 'order_received_timestamp'
4. All csv files have format: anon_quik_'month'_'year'.csv
### Notes:
1. All intermediate processed files are stored in processed_data_10_mins
2. The final processed files is generated in the main working directory as quik_data_all_final_10_mins.csv

### Helper Functions For Data Cleaning and Processing

In [63]:
def map_merchant_ids(input_csv, output_csv):
    # Define the mapping of merchant IDs to merchant names
    with open('merchant_id_map.json', 'r') as f:
        merchant_id_map = json.load(f)
    try:
        # Open input and output files
        with open(input_csv, 'r') as infile, open(output_csv, 'w', newline='') as outfile:
            reader = csv.reader(infile)
            writer = csv.writer(outfile)
            # Iterate through each row in the input file
            for row in reader:
                # Check if the first column (merchant ID) exists in the mapping
                if row[1] in merchant_id_map:
                    # Replace the merchant ID with the corresponding merchant name
                    row[1] = merchant_id_map[row[1]]
                else:
                    if row[1] == "merchant_id":
                        # Skip the header row
                        writer.writerow(row)
                        continue
                    # If the merchant ID is not found in the mapping, skip this row
                    print(f"Merchant ID {row[1]} not found in mapping.")
                    merchant_id_map[row[1]] = f"merchant_{len(merchant_id_map) + 1}"
                    f = open('merchant_id_map.json', 'w')
                    f.write(json.dumps(merchant_id_map))
                    f.close()
                    print(f"Added new mapping: {row[1]} -> {merchant_id_map[row[1]]}")
                    row[1] = merchant_id_map[row[1]]
                    print(f"Replaced merchant ID with new mapping: {row[1]}")


                # Write the modified row to the output file
                writer.writerow(row)

        print("Mapping completed successfully.")
        
    except FileNotFoundError:
        print("Error: File not found. Make sure 'quick_data.csv' exists in the current directory.")

    except Exception as e:
        print(f"An error occurred: {e}")
def drop_columns(input_csv, output_csv, columns_to_keep):
    df = pd.read_csv(input_csv)
    
    # Select only the specified columns to keep
    df_filtered = df[columns_to_keep]
    
    # check if directory does not exists
    if not os.path.exists(output_csv.split('/')[0]):
        os.makedirs(output_csv.split('/')[0])
    # Save the filtered DataFrame to a new CSV file
    df_filtered.to_csv(output_csv, index=False)
    print(f"Filtered CSV saved as {output_csv}")
def merge_csv_files(input_files, output_file):
    # Read all CSV files and combine them into a single DataFrame
    df_list = [pd.read_csv(file) for file in input_files]
    combined_df = pd.concat(df_list, ignore_index=True)
    
    # Save the combined DataFrame to a new CSV file
    combined_df.to_csv(output_file, index=False)
    print(f"Combined CSV saved as {output_file}")
def split_merchants():
    # Load the CSV data into a DataFrame
    data = pd.read_csv("quik_data_all.csv")

    # Get unique merchant IDs
    unique_merchant_ids = data['merchant_id'].unique()

    # Loop through unique merchant IDs and save data for each to a new CSV file
    i = 0
    for merchant_id in unique_merchant_ids:
        # Filter the data for the current merchant_id
        merchant_data = data[data['merchant_id'] == merchant_id]

        # Define the output filename using the merchant_id
        output_filename = f"separate_merchants/{merchant_id}.csv"

        # Save to CSV (optional: index=False to avoid including the index column in the output)
        merchant_data.to_csv(output_filename, index=False)
        i = i + 1

    print("Data has been split into separate files based on unique merchant_id values.")
def aggregate_orders_per_n_minutes(csv_file, output_csv, n_minutes):
    freq = f'{n_minutes}T'
    # Read the CSV file
    df = pd.read_csv(csv_file)
    
    # Convert the order_received_timestamp to a datetime object
    df['order_received_timestamp'] = pd.to_datetime(df['order_received_timestamp'])
    
    # Set the order_received_timestamp as the index
    df.set_index('order_received_timestamp', inplace=True)
    
    # Group by merchant_id and resample the data in 20-minute periods to count orders
    order_counts = df.groupby('merchant_id').resample(freq).size().reset_index(name='order_count')
    
    # Print the result
    print(order_counts)
    
    # Save the result to a new CSV file
    order_counts.to_csv(output_csv, index=False)



In [64]:
def process_data(time_window, data_dir):
    processed_files_dir = f'processed_data_{time_window}_mins'
    processed_files = []

    # check if directory does not exists
    if not os.path.exists(processed_files_dir):
        os.makedirs(processed_files_dir)

    for filename in os.listdir(data_dir):
        # Specify the input and output CSV file paths and the columns to keep
        input_csv = ''.join([data_dir, '/', filename])
        output_csv = input_csv.split('/')[1].replace('anon_', '')
        output_csv = ''.join([processed_files_dir, '/', output_csv])
        # print(f'input file: {input_csv}')
        # print(f'output file: {output_csv}')
        columns_to_keep = ['order_id', 'merchant_id', 'order_received_timestamp']  # Replace with your column names

        # Call the function to drop columns
        drop_columns(input_csv, output_csv, columns_to_keep)


        input_csv = output_csv
        tmp = input_csv.split('.')[0]
        output_csv = f'{tmp}_mapped.csv'
        # print(f'input file: {input_csv}')
        # print(f'output file: {output_csv}')

        # map merchant IDs to merchant names
        map_merchant_ids(input_csv, output_csv)

        input_csv = output_csv
        output_csv = output_csv.replace('_mapped', '_final')
        print(f'input file: {input_csv}')
        print(f'output file: {output_csv}')
        # Call the function to count orders
        aggregate_orders_per_n_minutes(input_csv, output_csv, time_window)
        processed_files.append(output_csv)
    dataset_file = f'quik_data_all_final_{time_window}_mins.csv'
    merge_csv_files(processed_files, dataset_file)
    # print("Inside the function: ")
    # print(dataset_file)
    return dataset_file



### Loading All Data


In [ ]:
dataset_file_name = process_data(10, "og_data") # process_data(time_window, data_dir) 
df = pd.read_csv(dataset_file_name) 
df.columns = df.columns.str.lower()
df['order_received_timestamp'] = pd.to_datetime(df['order_received_timestamp'], errors='coerce')


In [68]:
min(df['order_received_timestamp']), max(df['order_received_timestamp'])

(Timestamp('2023-12-01 00:00:00'), Timestamp('2024-02-29 23:50:00'))

In [69]:
df.head()

,merchant_id,order_received_timestamp,order_count
0,merchant_1,2023-12-01 00:10:00,1
1,merchant_1,2023-12-01 00:20:00,2
2,merchant_1,2023-12-01 00:30:00,0
3,merchant_1,2023-12-01 00:40:00,1
4,merchant_1,2023-12-01 00:50:00,2


In [70]:
agg_df = df.groupby(['order_received_timestamp', 'merchant_id']).agg({'order_count': 'sum'}).reset_index().sort_values(['merchant_id', 'order_received_timestamp']) #groups the dataframe


In [71]:
agg_df.head()

,order_received_timestamp,merchant_id,order_count
6,2023-12-01 00:10:00,merchant_1,1
13,2023-12-01 00:20:00,merchant_1,2
21,2023-12-01 00:30:00,merchant_1,0
29,2023-12-01 00:40:00,merchant_1,1
37,2023-12-01 00:50:00,merchant_1,2


In [72]:
agg_df.tail()

,order_received_timestamp,merchant_id,order_count
141349,2024-02-29 23:10:00,merchant_9,2
141358,2024-02-29 23:20:00,merchant_9,0
141366,2024-02-29 23:30:00,merchant_9,0
141374,2024-02-29 23:40:00,merchant_9,1
141380,2024-02-29 23:50:00,merchant_9,1


In [73]:
total_orders_df = agg_df.pivot(index='order_received_timestamp', columns='merchant_id', values='order_count')
total_orders_df.tail()

merchant_id,merchant_1,merchant_10,merchant_11,merchant_12,merchant_2,merchant_3,merchant_4,merchant_5,merchant_6,merchant_7,merchant_8,merchant_9
order_received_timestamp,,,,,,,,,,,,
2024-02-29 23:10:00,1.00,1.00,1.00,1.00,1.00,2.00,1.00,0.00,NaN,NaN,1.00,2.00
2024-02-29 23:20:00,1.00,1.00,0.00,NaN,1.00,1.00,2.00,0.00,NaN,NaN,0.00,0.00
2024-02-29 23:30:00,0.00,1.00,0.00,NaN,3.00,0.00,NaN,0.00,NaN,NaN,3.00,0.00
2024-02-29 23:40:00,1.00,2.00,0.00,NaN,1.00,1.00,NaN,0.00,NaN,NaN,0.00,1.00
2024-02-29 23:50:00,1.00,1.00,1.00,NaN,NaN,NaN,NaN,1.00,NaN,NaN,1.00,1.00


In [ ]:
# Data Visualization
for column in total_orders_df.columns:
    plt.figure(figsize=(10, 4))  
    plt.plot(total_orders_df.index, total_orders_df[column], marker='o', linestyle='-')  
    plt.title(f"Order Trend for {column}")
    plt.xlabel('Timestamp')
    plt.ylabel('Number of Orders')
    plt.grid(True)  
    plt.xticks(rotation=45)  
    plt.tight_layout()  
    plt.show()

In [77]:
start_date = total_orders_df.index.min()
total_orders_df = total_orders_df[total_orders_df.index >= start_date]

### Identifying and Removing Missing Data

In [81]:
def missing_data(input_data):
    '''
    This function returns dataframe with information about the percentage of nulls in each column and the column data type.
    
    input: pandas df
    output: pandas df
    
    '''
    
    total = input_data.isnull().sum()
    percent = (input_data.isnull().sum()/input_data.isnull().count()*100)
    table = pd.concat([total, percent], axis = 1, keys = ['Total', 'Percent'])
    types = []
    for col in input_data.columns: 
        dtype = str(input_data[col].dtype)
        types.append(dtype)
    table["Types"] = types
    return(pd.DataFrame(table))

In [80]:
missing_data(total_orders_df) #checks for null values for each merchant

,Total,Percent,Types
merchant_id,,,
merchant_1,52,0.40,float64
merchant_10,46,0.35,float64
merchant_11,6573,50.16,float64
merchant_12,9147,69.80,float64
merchant_2,2,0.02,float64
merchant_3,1,0.01,float64
merchant_4,9,0.07,float64
merchant_5,2,0.02,float64
merchant_6,10,0.08,float64


In [82]:
total_orders_df.fillna(0, inplace=True) #filling nan values with zeros
#assuming there was no orders at that 20 minute period for that merchant

missing_data(total_orders_df) #check missing values again to make sure there's none

,Total,Percent,Types
merchant_id,,,
merchant_1,0,0.00,float64
merchant_10,0,0.00,float64
merchant_11,0,0.00,float64
merchant_12,0,0.00,float64
merchant_2,0,0.00,float64
merchant_3,0,0.00,float64
merchant_4,0,0.00,float64
merchant_5,0,0.00,float64
merchant_6,0,0.00,float64


#### This section groups merchants by the volume of data. The higher volume data the better for forecasting since we can forecast past the noise and end up with lower error bars. This helped understand why some merchats had higher error bars than other merchants (['merchant_12', 'merchant_11', 'merchant_7', 'merchant_6'])

### Category Grouping

In [83]:
avg_daily_orders = total_orders_df.apply(np.mean, axis=0).sort_values() #typical order volume per merchant across all timestamps.

low, mid = np.percentile(avg_daily_orders, [33, 66]) #thresholds for low, mid, and high order volumes.


In [84]:
# Create lists of merchants based on order volume categories
low_vol_columns = list(avg_daily_orders[avg_daily_orders <= low].index)
mid_vol_columns = list(avg_daily_orders[(avg_daily_orders > low) & (avg_daily_orders < mid)].index)
high_vol_columns = list(avg_daily_orders[avg_daily_orders >= mid].index)



In [85]:
high_vol_columns

['merchant_10', 'merchant_1', 'merchant_4', 'merchant_5']

In [86]:
low_vol_columns

['merchant_12', 'merchant_11', 'merchant_7', 'merchant_6']

### Visualizing low, mid, high columns

In [ ]:

# plt.figure(figsize=(12, 6))  
# for merchant in low_vol_columns:
#     plt.plot(total_orders_df.index, total_orders_df[merchant], label=merchant)

# plt.title('Order Trends for Low Volume Merchants')
# plt.xlabel('Timestamp')
# plt.ylabel('Number of Orders')
# plt.legend(title='Merchant', loc='best')  
# plt.grid(True)  
# plt.xticks(rotation=45)  
# plt.tight_layout() 
# plt.show()

In [ ]:
# plt.figure(figsize=(12, 6)) 
# for merchant in mid_vol_columns:
#     plt.plot(total_orders_df.index, total_orders_df[merchant], label=merchant)

# plt.title('Order Trends for Mid Volume Merchants')
# plt.xlabel('Timestamp')
# plt.ylabel('Number of Orders')
# plt.legend(title='Merchant', loc='best', bbox_to_anchor=(1.1, 1.05))  # Positioning the legend outside the plot area
# plt.grid(True)  # Adding a grid for better readability
# plt.xticks(rotation=30)  
# plt.tight_layout() 
# plt.show()

In [ ]:
# plt.figure(figsize=(12, 6))  # Set a suitable figure size
# for merchant in high_vol_columns:
#     plt.plot(total_orders_df.index, total_orders_df[merchant], label=merchant)

# plt.title('Order Trends for High Volume Merchants')
# plt.xlabel('Timestamp')
# plt.ylabel('Number of Orders')
# plt.legend(title='Merchant', loc='best', bbox_to_anchor=(1.1, 1.05))  # Properly title the legend and adjust its position
# plt.xticks(rotation=30)
# plt.grid(True)  # Optionally add a grid
# plt.tight_layout()  # Adjust layout
# plt.show()

### Model Latest Checkpoint 

In [97]:
import os
from pathlib import Path
from datetime import datetime
import pickle

def load_most_recent_checkpoint(directory: str) -> object:
    """
    Load the most recent Prophet model checkpoint from the specified directory.

    Args:
    - directory (str): Path to the directory containing the checkpoint files.

    Returns:
    - object: The loaded Prophet model from the most recent checkpoint.
    """
    directory_path = Path(directory)

     # Create the directory if it doesn't exist
    if not directory_path.exists():
        directory_path.mkdir(parents=True, exist_ok=True)
        print(f"Directory created: {directory}")
        return None  # Return None as there's no checkpoint to load

    # List all .pkl files in the directory
    pkl_files = list(directory_path.glob("prophet_model_*.pkl"))

    if not pkl_files:
        print("No .pkl files found in the directory.")
        return None

    # Extract dates from filenames and sort by date (newest first)
    def extract_date_from_filename(file):
        date_str = file.stem.split('_')[-1]  # Extract the date part from the filename
        return datetime.strptime(date_str, '%Y-%m-%d')  # Convert to datetime object

    # Sort files by extracted date
    pkl_files.sort(key=lambda file: extract_date_from_filename(file), reverse=True)

    # Get the most recent file
    most_recent_file = pkl_files[0]

    print(f"Most recent model checkpoint: {most_recent_file}")

    # Load the most recent model checkpoint (assuming it’s a pickle file)
    with open(most_recent_file, 'rb') as f:
        model = pickle.load(f)

    return model


### HYPERPARAMETER TUNING AND BACKTESTING PIPELINE

Ranges give on Prophet's documentation

1. changepoint_prior_scale [0.001, 0.5] 
2. seasonality_prior_scale [0.01, 10]
3. holidays_prior_scale [0.01, 10] 
4. seasonality_mode ['additive', 'multiplicative'].
5. changepoint_range [0.5, 0.95]

https://facebook.github.io/prophet/docs/diagnostics.html

In [88]:
changepoint_prior_scale_range = np.linspace(0.001, 0.5, num=5).tolist()
seasonality_prior_scale_range = np.linspace(0.01, 10, num=5).tolist()
holidays_prior_scale_range = np.linspace(0.01, 10, num=5).tolist()
seasonality_mode_options = ['additive', 'multiplicative']
changepoint_range_range = list(np.linspace(0.5, 0.95, num=5))

In [ ]:
# Tuning for the above five hyperparameters (takes more than 24 hours running)
# Already done this step and stored the optimized values in a dictionary below

# start_time = time.time()

# dicts = {}

# for feature in total_orders_df.columns:
  
#     category_df = total_orders_df[feature].copy().reset_index()
#     category_df.columns = ["ds", "y"]

#     category_df[["y"]] = category_df[["y"]].apply(pd.to_numeric)
#     category_df["ds"] = pd.to_datetime(category_df["ds"])
    
#     param_grid = {  
#         "changepoint_prior_scale": changepoint_prior_scale_range,
#         "seasonality_prior_scale": seasonality_prior_scale_range,
#         'holidays_prior_scale': holidays_prior_scale_range,
#         'seasonality_mode': seasonality_mode_options,
#         'changepoint_range': changepoint_range_range}

#     # Generate all combinations of parameters
#     all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
#     rmses = []
#     maes = []

#     # Use cross validation to evaluate all parameters
#     for params in all_params:
#         m = Prophet(**params).fit(category_df)  # Fit model with given params
#         df_cv = cross_validation(m, initial="60 days", period="5 days", horizon = "1 days") 
#         df_p = performance_metrics(df_cv, rolling_window=1)
#         print(df_p)
#         rmses.append(df_p["rmse"].values[0])
#         maes.append(df_p["mae"].values[0])

#     # Find the best parameters
#     tuning_results = pd.DataFrame(all_params)
#     tuning_results["rmse"] = rmses
#     tuning_results["mae"] = maes

#     params_dict = dict(tuning_results.sort_values("rmse").reset_index(drop=True).iloc[0])
#     params_dict["column"] = feature 
    
#     dicts[feature] = params_dict

# print("--- %s seconds ---" % (time.time() - start_time)) 

Here is a dictionary of the optimized parameters for each merchant. 

In [ ]:
# dicts

In [89]:
dicts = {'merchant_1': {'changepoint_prior_scale': 0.001,
  'seasonality_prior_scale': 7.5024999999999995,
  'holidays_prior_scale': 10.0,
  'seasonality_mode': 'multiplicative',
  'changepoint_range': 0.5,
  'rmse': 3.4315968192683637,
  'mae': 2.4984342328440046,
  'column': 'merchant_1'},
 'merchant_10': {'changepoint_prior_scale': 0.12575,
  'seasonality_prior_scale': 5.005,
  'holidays_prior_scale': 10.0,
  'seasonality_mode': 'multiplicative',
  'changepoint_range': 0.6125,
  'rmse': 3.5954550911116465,
  'mae': 2.5557456620858,
  'column': 'merchant_10'},
 'merchant_11': {'changepoint_prior_scale': 0.2505,
  'seasonality_prior_scale': 7.5024999999999995,
  'holidays_prior_scale': 7.5024999999999995,
  'seasonality_mode': 'multiplicative',
  'changepoint_range': 0.6125,
  'rmse': 1.7238502662395094,
  'mae': 1.299743933957384,
  'column': 'merchant_11'},
 'merchant_12': {'changepoint_prior_scale': 0.37525,
  'seasonality_prior_scale': 0.01,
  'holidays_prior_scale': 7.5024999999999995,
  'seasonality_mode': 'multiplicative',
  'changepoint_range': 0.95,
  'rmse': 2.6299807073691883,
  'mae': 2.0098218657752147,
  'column': 'merchant_12'},
 'merchant_2': {'changepoint_prior_scale': 0.001,
  'seasonality_prior_scale': 5.005,
  'holidays_prior_scale': 10.0,
  'seasonality_mode': 'multiplicative',
  'changepoint_range': 0.725,
  'rmse': 3.0502628448731963,
  'mae': 2.221268396746129,
  'column': 'merchant_2'},
 'merchant_3': {'changepoint_prior_scale': 0.001,
  'seasonality_prior_scale': 5.005,
  'holidays_prior_scale': 0.01,
  'seasonality_mode': 'additive',
  'changepoint_range': 0.5,
  'rmse': 2.361647507971563,
  'mae': 1.7546935832409052,
  'column': 'merchant_3'},
 'merchant_4': {'changepoint_prior_scale': 0.001,
  'seasonality_prior_scale': 2.5075,
  'holidays_prior_scale': 0.01,
  'seasonality_mode': 'multiplicative',
  'changepoint_range': 0.5,
  'rmse': 3.709540232072044,
  'mae': 2.6422456357755504,
  'column': 'merchant_4'},
 'merchant_5': {'changepoint_prior_scale': 0.5,
  'seasonality_prior_scale': 2.5075,
  'holidays_prior_scale': 0.01,
  'seasonality_mode': 'multiplicative',
  'changepoint_range': 0.95,
  'rmse': 3.2533279397777206,
  'mae': 2.3681656901119186,
  'column': 'merchant_5'},
 'merchant_6': {'changepoint_prior_scale': 0.001,
  'seasonality_prior_scale': 2.5075,
  'holidays_prior_scale': 2.5075,
  'seasonality_mode': 'additive',
  'changepoint_range': 0.8374999999999999,
  'rmse': 2.0234077366020458,
  'mae': 1.5088789734778578,
  'column': 'merchant_6'},
 'merchant_7': {'changepoint_prior_scale': 0.001,
  'seasonality_prior_scale': 0.01,
  'holidays_prior_scale': 0.01,
  'seasonality_mode': 'additive',
  'changepoint_range': 0.5,
  'rmse': 2.0137982163020816,
  'mae': 1.4871130296584607,
  'column': 'merchant_7'},
 'merchant_8': {'changepoint_prior_scale': 0.001,
  'seasonality_prior_scale': 5.005,
  'holidays_prior_scale': 7.5024999999999995,
  'seasonality_mode': 'multiplicative',
  'changepoint_range': 0.725,
  'rmse': 3.0955259810842684,
  'mae': 2.3027168995176646,
  'column': 'merchant_8'},
 'merchant_9': {'changepoint_prior_scale': 0.001,
  'seasonality_prior_scale': 10.0,
  'holidays_prior_scale': 5.005,
  'seasonality_mode': 'multiplicative',
  'changepoint_range': 0.5,
  'rmse': 3.2519446602474082,
  'mae': 2.336450967142641,
  'column': 'merchant_9'}}

### HOLIDAY DATA To Fetch UAE Hollidays

In [90]:
# Initialize the holiday DataFrame
holiday = pd.DataFrame([])

# Fetch holidays for UAE for the years 2023 and 2024
for date_, name in sorted(holidays.AE(years=[2023, 2024]).items()):
    holiday = pd.concat([holiday, pd.DataFrame({'ds': [date_], 'holiday': ["UAE-Holidays"], 'lower_window': [-2], 'upper_window': [1]})], ignore_index=True)

# Manually add December 30 and 31 holidays
# We can add other desired hollidays manually if not in the holiday DataFrame
manual_holidays = pd.DataFrame({
    'ds': ['2023-12-30', '2023-12-31'],
    'holiday': ['Manual-Holiday', 'Manual-Holiday'],
    'lower_window': [-2, -2],
    'upper_window': [1, 1]
})

# Convert 'ds' to datetime
manual_holidays['ds'] = pd.to_datetime(manual_holidays['ds'])

# Append manual holidays to the holiday DataFrame
holiday = pd.concat([holiday, manual_holidays], ignore_index=True)

# Convert 'ds' to datetime
holiday['ds'] = pd.to_datetime(holiday['ds'], format='%Y-%m-%d', errors='ignore')

# Display the updated holiday DataFrame
print(holiday.head(10))
print(holiday.tail(10))


          ds       holiday  lower_window  upper_window
0 2023-01-01  UAE-Holidays            -2             1
1 2023-04-20  UAE-Holidays            -2             1
2 2023-04-21  UAE-Holidays            -2             1
3 2023-04-22  UAE-Holidays            -2             1
4 2023-04-23  UAE-Holidays            -2             1
5 2023-06-27  UAE-Holidays            -2             1
6 2023-06-28  UAE-Holidays            -2             1
7 2023-06-29  UAE-Holidays            -2             1
8 2023-06-30  UAE-Holidays            -2             1
9 2023-07-21  UAE-Holidays            -2             1
           ds         holiday  lower_window  upper_window
18 2024-06-15    UAE-Holidays            -2             1
19 2024-06-16    UAE-Holidays            -2             1
20 2024-06-17    UAE-Holidays            -2             1
21 2024-06-18    UAE-Holidays            -2             1
22 2024-07-07    UAE-Holidays            -2             1
23 2024-09-15    UAE-Holidays            -2    

C:\Users\Nourhan\AppData\Local\Temp\ipykernel_25360\3431846932.py:24: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  holiday['ds'] = pd.to_datetime(holiday['ds'], format='%Y-%m-%d', errors='ignore')


### BACKTESTING WITH TUNED METRICS

In [91]:
prediction_days = 1 # Forecast for the next day only
forecast_start_date = max(total_orders_df.index) - timedelta(prediction_days) 

In [92]:
forecast_start_date

Timestamp('2024-02-28 23:50:00')

In [ ]:
import os
import matplotlib.pyplot as plt


# Lists to store RMSE, MAE, and average orders per 20-minute interval for each merchant
rmse_list = []
mae_list = []
average_orders_list = []

forecasted_dfs = []

# Create directory to save plots
results_dir = 'results_10_mins'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)


excel_path = os.path.join(results_dir, 'predicted_forecasts.xlsx')
writer = pd.ExcelWriter(excel_path, engine='openpyxl')


for feature in total_orders_df.columns:
    
    # Formatting
    df_copy = total_orders_df[feature].copy().reset_index()
    df_copy.columns = ['ds', 'y']
    df_copy[['y']] = df_copy[['y']].apply(pd.to_numeric)
    df_copy['ds'] = pd.to_datetime(df_copy['ds'])
    
    df_copy_ = df_copy[df_copy['ds'] < forecast_start_date]
    
    # Finding the right params_dict for this category
    params_dict = dicts[feature]
    
    m = load_most_recent_checkpoint("prophet_checkpoints")

    #If there is no checkpoint (first time)
    if m is None:
        m = Prophet(
        changepoint_prior_scale=params_dict['changepoint_prior_scale'],
        seasonality_prior_scale=params_dict['seasonality_prior_scale'],
        holidays_prior_scale=params_dict['holidays_prior_scale'],
        seasonality_mode=params_dict['seasonality_mode'],
        changepoint_range=params_dict['changepoint_range'],
        holidays=holiday
        )

    m.fit(df_copy_)

    future = m.make_future_dataframe(periods=prediction_days * 144, freq='10min')
    fcst_prophet_train = m.predict(future)
    
    filter = fcst_prophet_train['ds'] >= forecast_start_date 
    predicted_df = fcst_prophet_train[filter][['ds', 'yhat']]
    predicted_df = predicted_df.merge(df_copy, on='ds')

    # Round up predictions
    predicted_df['yhat'] = np.ceil(predicted_df['yhat'])

    #print(predicted_df.head(20))
    # Calculate RMSE and MAE
    rmse = np.sqrt(np.mean((predicted_df['y'] - predicted_df['yhat'])**2))
    mae = np.mean(np.abs(predicted_df['y'] - predicted_df['yhat']))

    # Append RMSE and MAE to the lists
    rmse_list.append(rmse)
    mae_list.append(mae)

    # Calculate and store the average number of orders per 20-minute interval
    average_orders = np.mean(predicted_df['y'])
    average_orders_list.append(average_orders)

    print(f"{feature} - RMSE: {rmse:.2f}, MAE: {mae:.2f}, Average Orders per 10min: {average_orders:.2f}")
    
    # Plotting
    plt.figure(figsize=(12, 6))
    plt.plot(predicted_df['ds'], predicted_df['y'], label='Actual')
    plt.plot(predicted_df['ds'], predicted_df['yhat'], label='Predicted', linestyle='--')
    plt.title(f'Actual vs Predicted for {feature}')
    plt.xlabel('Date')
    plt.ylabel('Orders')
    plt.legend()
    plt.grid(True)

    # Save the plot to the results directory
    plot_path = os.path.join(results_dir, f'{feature}_forecast.png')
    plt.savefig(plot_path)
    plt.close()
    # Save the predicted DataFrame to the Excel file
    predicted_df.to_excel(writer, sheet_name=feature, index=False)

# Finalize and save the Excel file
writer._save()
writer.close()

# Calculate average RMSE, MAE, and average orders across all merchants
average_rmse = np.mean(rmse_list)
average_mae = np.mean(mae_list)
average_orders_across_merchants = np.mean(average_orders_list)

print(f"Average RMSE across all merchants: {average_rmse:.2f}")
print(f"Average MAE across all merchants: {average_mae:.2f}")
print(f"Average Orders per 10 min across all merchants: {average_orders_across_merchants:.2f}")


In [ ]:
#predicted_df


### Saving the model

In [100]:
import pickle
from datetime import datetime

# Get the current date
current_date = datetime.now().strftime('%Y-%m-%d')

# Save the model with the current date in the filename
model_filename = f'prophet_checkpoints/prophet_model_{current_date}.pkl'
with open(model_filename, 'wb') as f:
    pickle.dump(m, f)
